In [1]:
%pip install pandas
%pip install scikit-learn
%pip install matplotlib
%pip install seaborn
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 2.8 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 4.9 MB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 2.5 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 3.0 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.2 MB/s eta 0:00:00 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_log_error, mean_absolute_error


import seaborn as sns
import matplotlib.pyplot as plt

import xgboost as xgb

In [2]:
data = pd.read_csv('./Train.csv')
data.head()

/tmp/ipykernel_14270/2216384314.py:1: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./Train.csv')


,id,realSum,room_type,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,lng,lat,city,day_type
0,1,210.3658537,Entire home/apt,FALSE,FALSE,2,FALSE,0,1,9,91,0,0.541021778,0.42791926,-9.13395,38.71038,lisbon,weekday
1,2,176.1817209,Private room,FALSE,TRUE,2,FALSE,1,0,9,89,1,3.032839151,0.343492056,2.14165,41.37011,barcelona,weekday
2,3,142.0541405,Private room,FALSE,TRUE,2,TRUE,0,0,10,98,1,11.90971188,6.710908979,0.00475,51.44025,london,weekend
3,4,428.7445242,Entire home/apt,FALSE,FALSE,4,FALSE,0,0,9,94,0,4.158389765,0.145570338,2.30179,48.87297,paris,weekend
4,5,220.2798022,Entire home/apt,FALSE,FALSE,6,TRUE,0,1,10,93,2,1.302189968,0.369474779,23.72468,37.96746,athens,weekend


In [ ]:
# remove bad row
data = data[data['id'] != 2376]

# Normalize the price
data['realSum'] = data['realSum'].astype(float).round().astype(int)
data['log_price'] = np.log1p(data['realSum'])

/tmp/ipykernel_14270/2180715776.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['room_shared'] = data['room_shared'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})
/tmp/ipykernel_14270/2180715776.py:10: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['room_private'] = data['room_private'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})
/tmp/ipykernel_14270/2180715776.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explic

In [82]:
def preprocessor (data):
  # Convert 'FALSE' and 'TRUE' strings to 0 and 1, and booleans False and True to 0 and 1 for "room_shared" and "room_private"
  data['room_shared'] = data['room_shared'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})
  data['room_private'] = data['room_private'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})
  data['host_is_superhost'] = data['room_private'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})

  # Convert '0' and '1' strings to numbers for "multi" and "biz", and leave numbers as they are
  data['multi'] = data['multi'].replace({'0': 0, '1': 1}).astype(int)
  data['biz'] = data['biz'].replace({'0': 0, '1': 1}).astype(int)
  # convert numeric vals that are type string to int
  data['person_capacity'] = data['person_capacity'].astype(int)
  data['cleanliness_rating'] = data['cleanliness_rating'].astype(int)
  data['guest_satisfaction_overall'] = data['guest_satisfaction_overall'].astype(int)
  data['bedrooms'] = data['bedrooms'].astype(int)
  data['dist'] = data['dist'].astype(float)
  data['metro_dist'] = data['metro_dist'].astype(float)
  data['lng'] = data['lng'].astype(float)
  data['lat'] = data['lat'].astype(float)

  # one-hot encoding categorical features
  data = pd.get_dummies(data, columns=['room_type'], drop_first=True)
  data = pd.get_dummies(data, columns=['city'], drop_first=True)
  data = pd.get_dummies(data, columns=['day_type'], drop_first=True)

  return data

In [ ]:
data = preprocessor(data)

In [84]:
X = data.drop(columns=["log_price", "realSum", "id"])
y = data["log_price"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [85]:
xgb_model = xgb.XGBRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    tree_method="hist",
)

In [86]:
xgb_model.fit(
  X_train,
  y_train
)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [87]:
y_pred_log = xgb_model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

In [88]:
epsilon = 1e-6
y_pred = np.clip(y_pred, epsilon, None)
y_true = np.clip(y_true, epsilon, None)

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_log_error(y_true, y_pred))

print(f"MAE:  {mae:.2f}")
print(f"RMSLE: {rmse:.4f}")

MAE:  58.11
RMSLE: 0.2583


In [89]:
# test with all the dataset
y_pred_log_all = xgb_model.predict(X)
y_pred_all = np.expm1(y_pred_log_all)
y_true_all = np.expm1(data["log_price"])
y_pred_all = np.clip(y_pred_all, epsilon, None)
y_true_all = np.clip(y_true_all, epsilon, None)

mae_all = mean_absolute_error(y_true_all, y_pred_all)
rmse_all = np.sqrt(mean_squared_log_error(y_true_all, y_pred_all))
print(f"MAE:  {mae_all:.2f}")
print(f"RMSLE: {rmse_all:.4f}")

MAE:  32.16
RMSLE: 0.1535


In [90]:
# Load the test dataset
test_data = pd.read_csv('./Test.csv')

# Preprocess the test dataset using the same preprocessor
test_data = preprocessor(test_data)

# Predict log prices and convert them back to real prices
y_pred_log_test = xgb_model.predict(test_data.drop(columns=["id"]))
y_pred_test = np.expm1(y_pred_log_test)

# Create the output dataframe
output = pd.DataFrame({
  "id": test_data["id"],
  "realSum": y_pred_test
})

# Save the result to a CSV file
output.to_csv("predicted_results.csv", index=False)

/tmp/ipykernel_14270/181979679.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['room_shared'] = data['room_shared'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})
/tmp/ipykernel_14270/181979679.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['room_private'] = data['room_private'].replace({'FALSE': 0, 'TRUE': 1, False: 0, True: 1})


In [91]:
# Retrain the model on the entire dataset
xgb_model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=1000,
             n_jobs=None, num_parallel_tree=None, ...)

In [92]:
# test with all the dataset
y_pred_log_all = xgb_model.predict(X)
y_pred_all = np.expm1(y_pred_log_all)
y_true_all = np.expm1(data["log_price"])
y_pred_all = np.clip(y_pred_all, epsilon, None)
y_true_all = np.clip(y_true_all, epsilon, None)

mae_all = mean_absolute_error(y_true_all, y_pred_all)
rmse_all = np.sqrt(mean_squared_log_error(y_true_all, y_pred_all))
print(f"MAE:  {mae_all:.2f}")
print(f"RMSLE: {rmse_all:.4f}")

MAE:  28.86
RMSLE: 0.1255


In [93]:
# Predict log prices and convert them back to real prices
y_pred_log_test = xgb_model.predict(test_data.drop(columns=["id"]))
y_pred_test = np.expm1(y_pred_log_test)

# Create the output dataframe
output = pd.DataFrame({
  "id": test_data["id"],
  "realSum": y_pred_test
})

# Save the result to a CSV file
output.to_csv("predicted_results2.csv", index=False)

In [97]:
# Find best params with GridSearchCV

param_grid = {
    'n_estimators': [1000, 2000, 3000],
    'max_depth': [6, 8, 12],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

model = xgb.XGBRegressor(tree_method="hist", random_state=42)

grid = GridSearchCV(
    model,
    param_grid=param_grid,
    scoring='neg_mean_squared_log_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid.fit(X, y)
print("Best parameters found: ", grid.best_params_)
print("Best score: ", grid.best_score_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters found:  {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 12, 'n_estimators': 3000, 'subsample': 0.8}
Best score:  -0.0015501386258660332


In [98]:
newModel = xgb.XGBRegressor(
  colsample_bytree= 0.8,
  learning_rate=0.01,
  max_depth=12,
  n_estimators=3000,
  subsample=0.8
)

newModel.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.01, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=3000,
             n_jobs=None, num_parallel_tree=None, ...)

In [99]:
# test with all the dataset
y_pred_log_all = newModel.predict(X)
y_pred_all = np.expm1(y_pred_log_all)
y_true_all = np.expm1(data["log_price"])
y_pred_all = np.clip(y_pred_all, epsilon, None)
y_true_all = np.clip(y_true_all, epsilon, None)
mae_all = mean_absolute_error(y_true_all, y_pred_all)
rmse_all = np.sqrt(mean_squared_log_error(y_true_all, y_pred_all))
print(f"MAE:  {mae_all:.2f}")
print(f"RMSLE: {rmse_all:.4f}")

# Predict log prices and convert them back to real prices
y_pred_log_test = newModel.predict(test_data.drop(columns=["id"]))
y_pred_test = np.expm1(y_pred_log_test)
# Create the output dataframe
output = pd.DataFrame({
  "id": test_data["id"],
  "realSum": y_pred_test
})
# Save the result to a CSV file
output.to_csv("predicted_results3.csv", index=False)

MAE:  7.72
RMSLE: 0.0371


In [100]:
X.head()

,room_shared,room_private,person_capacity,host_is_superhost,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,...,city_athens,city_barcelona,city_berlin,city_budapest,city_lisbon,city_london,city_paris,city_rome,city_vienna,day_type_weekend
0,0,0,2,0,0,1,9,91,0,0.541022,...,False,False,False,False,True,False,False,False,False,False
1,0,1,2,1,1,0,9,89,1,3.032839,...,False,True,False,False,False,False,False,False,False,False
2,0,1,2,1,0,0,10,98,1,11.909712,...,False,False,False,False,False,True,False,False,False,True
3,0,0,4,0,0,0,9,94,0,4.158390,...,False,False,False,False,False,False,True,False,False,True
4,0,0,6,0,0,1,10,93,2,1.302190,...,True,False,False,False,False,False,False,False,False,True
